## Introduction to Parallel Programming

### Motivation for this Class

* Parallelism is everywhere!
    * Multicore, GPU, cloud, HPC
    * Every program/programmer needs to address it
* Traditional CS curriculum totally misses the point
    * Model the world as serial algorithms
    
### Who should take this course?

* Anyone who needs to program ever. Period! Exclamation point!! …  
* Designed for Undergraduates in CS: 
    * quick lift of skills good for employers and internships
* Suitable for graduate students in Science and Engineering
    * Mimimize 

